# Solar Panel Performance Optimization: A Comprehensive Analysis

## Overview

This notebook provides a complete analysis of solar photovoltaic (PV) system performance using real-world operational data. We'll explore key renewable energy concepts and calculate industry-standard metrics to evaluate system efficiency and economic viability.

### Dataset Description

- **Time period**: 1 year of continuous operation (8,760 hourly observations)
- **System capacity**: 50 kW nameplate capacity (rated power output)
- **Data granularity**: Hourly measurements
- **Variables**: Solar irradiance, ambient temperature, panel temperature, power output, electricity prices

### System Specifications

- **Nameplate capacity**: 50 kW (maximum rated output under standard test conditions)
- **Base efficiency**: 18% (portion of solar energy converted to electricity)
- **Temperature coefficient**: -0.4%/°C (efficiency reduction per degree above 25°C)

### Renewable Energy Concepts Covered

1. **Solar Irradiance**: Measure of solar power density from the sun (W/m²)
2. **Capacity Factor**: Ratio of actual energy output to theoretical maximum production
3. **Performance Ratio**: System efficiency accounting for real-world losses
4. **Temperature Effects**: Impact of heat on PV panel efficiency
5. **Energy Yield**: Total annual electricity generation (kWh/year)
6. **Peak Sun Hours**: Equivalent daily hours of 1,000 W/m² irradiance
7. **LCOE**: Levelized Cost of Energy (lifetime cost per kWh produced)
8. **Time-of-Use Economics**: Value optimization based on dynamic electricity pricing

### Learning Objectives

- Understand solar PV system performance characteristics
- Calculate industry-standard renewable energy metrics
- Identify optimization opportunities for maximum efficiency
- Evaluate economic viability and return on investment
- Analyze temporal patterns in solar energy production

## 1. Setup and Configuration

Import required libraries for data analysis, visualization, and calculations.

In [ ]:
# Data manipulation and analysis
# Date and time handling

# Visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Statistical analysis

# Configure visualization settings
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 10

# Display settings for pandas
pd.set_option("display.max_columns", None)
pd.set_option("display.precision", 2)

# System specifications
NAMEPLATE_CAPACITY_KW = 50  # Rated power output
BASE_EFFICIENCY = 0.18  # 18% conversion efficiency
TEMP_COEFFICIENT = -0.004  # -0.4% per degree Celsius
REFERENCE_TEMP_C = 25  # Standard test condition temperature
STANDARD_IRRADIANCE = 1000  # W/m² (standard test condition)

print("Libraries imported successfully!")
print(f"System Configuration: {NAMEPLATE_CAPACITY_KW} kW nameplate capacity")
print(f"Base Efficiency: {BASE_EFFICIENCY * 100}%")
print(f"Temperature Coefficient: {TEMP_COEFFICIENT * 100}%/°C")

## 2. Load and Explore Data

Load the solar panel operational data and perform initial exploratory analysis.

In [ ]:
# Load the dataset
df = pd.read_csv("solar_panel_data.csv")

# Parse datetime column
df["datetime"] = pd.to_datetime(df["datetime"])
df = df.set_index("datetime")

# Create additional time-based features
df["date"] = df.index.date
df["day_of_year"] = df.index.dayofyear
df["week"] = df.index.isocalendar().week
df["season"] = df["month"].map(
    {
        12: "Winter",
        1: "Winter",
        2: "Winter",
        3: "Spring",
        4: "Spring",
        5: "Spring",
        6: "Summer",
        7: "Summer",
        8: "Summer",
        9: "Fall",
        10: "Fall",
        11: "Fall",
    }
)

print("Dataset Shape:", df.shape)
print("\nTime Period:", df.index.min(), "to", df.index.max())
print(f"\nTotal Hours of Data: {len(df):,}")
print("\n" + "=" * 60)
print("DATASET OVERVIEW")
print("=" * 60)
df.head(10)

In [ ]:
# Basic statistical summary
print("=" * 60)
print("DESCRIPTIVE STATISTICS")
print("=" * 60)
print(
    df[
        [
            "irradiance_w_m2",
            "ambient_temp_c",
            "panel_temp_c",
            "power_output_kw",
            "electricity_price_kwh",
        ]
    ].describe()
)

# Check for missing values
print("\n" + "=" * 60)
print("DATA QUALITY CHECK")
print("=" * 60)
print("Missing values per column:")
print(df.isnull().sum())

# Data completeness
completeness = (1 - df.isnull().sum() / len(df)) * 100
print(f"\nOverall Data Completeness: {completeness.mean():.2f}%")

## 3. Solar Irradiance Patterns

**Solar irradiance** is the power per unit area received from the Sun, measured in watts per square meter (W/m²). It's the fundamental driver of solar panel electricity generation. Understanding irradiance patterns helps optimize system design and predict energy production.

In [ ]:
# Daily irradiance patterns (average by hour)
hourly_irradiance = df.groupby("hour")["irradiance_w_m2"].agg(["mean", "std", "max"])

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Average daily irradiance pattern
ax1 = axes[0, 0]
ax1.plot(
    hourly_irradiance.index,
    hourly_irradiance["mean"],
    linewidth=2.5,
    marker="o",
    markersize=6,
    color="#FF6B35",
    label="Mean",
)
ax1.fill_between(
    hourly_irradiance.index,
    hourly_irradiance["mean"] - hourly_irradiance["std"],
    hourly_irradiance["mean"] + hourly_irradiance["std"],
    alpha=0.3,
    color="#FF6B35",
    label="±1 Std Dev",
)
ax1.axhline(
    y=STANDARD_IRRADIANCE,
    color="red",
    linestyle="--",
    linewidth=2,
    label="Standard Test Condition (1000 W/m²)",
)
ax1.set_xlabel("Hour of Day", fontsize=12, fontweight="bold")
ax1.set_ylabel("Solar Irradiance (W/m²)", fontsize=12, fontweight="bold")
ax1.set_title("Average Daily Irradiance Pattern", fontsize=14, fontweight="bold")
ax1.grid(True, alpha=0.3)
ax1.legend(loc="upper left")
ax1.set_xlim(0, 23)

# Plot 2: Seasonal irradiance patterns
ax2 = axes[0, 1]
for season in ["Winter", "Spring", "Summer", "Fall"]:
    season_data = df[df["season"] == season].groupby("hour")["irradiance_w_m2"].mean()
    ax2.plot(
        season_data.index, season_data.values, linewidth=2.5, marker="o", markersize=5, label=season
    )
ax2.set_xlabel("Hour of Day", fontsize=12, fontweight="bold")
ax2.set_ylabel("Solar Irradiance (W/m²)", fontsize=12, fontweight="bold")
ax2.set_title("Seasonal Irradiance Patterns", fontsize=14, fontweight="bold")
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_xlim(0, 23)

# Plot 3: Monthly average irradiance
ax3 = axes[1, 0]
monthly_irradiance = df.groupby("month")["irradiance_w_m2"].mean()
colors = plt.cm.YlOrRd(np.linspace(0.3, 0.9, 12))
bars = ax3.bar(monthly_irradiance.index, monthly_irradiance.values, color=colors, edgecolor="black")
ax3.set_xlabel("Month", fontsize=12, fontweight="bold")
ax3.set_ylabel("Average Irradiance (W/m²)", fontsize=12, fontweight="bold")
ax3.set_title("Monthly Average Irradiance", fontsize=14, fontweight="bold")
ax3.set_xticks(range(1, 13))
ax3.set_xticklabels(
    ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
)
ax3.grid(True, alpha=0.3, axis="y")

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax3.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{height:.0f}",
        ha="center",
        va="bottom",
        fontsize=9,
    )

# Plot 4: Irradiance heatmap by hour and month
ax4 = axes[1, 1]
irradiance_pivot = df.pivot_table(
    values="irradiance_w_m2", index="hour", columns="month", aggfunc="mean"
)
sns.heatmap(
    irradiance_pivot,
    cmap="YlOrRd",
    annot=False,
    fmt=".0f",
    cbar_kws={"label": "Irradiance (W/m²)"},
    ax=ax4,
)
ax4.set_xlabel("Month", fontsize=12, fontweight="bold")
ax4.set_ylabel("Hour of Day", fontsize=12, fontweight="bold")
ax4.set_title("Irradiance Heatmap: Hour vs Month", fontsize=14, fontweight="bold")
ax4.set_xticklabels(
    ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"], rotation=0
)

plt.tight_layout()
plt.show()

# Calculate peak production times
print("=" * 60)
print("PEAK IRRADIANCE ANALYSIS")
print("=" * 60)
peak_hour = hourly_irradiance["mean"].idxmax()
peak_value = hourly_irradiance["mean"].max()
print(f"Peak Irradiance Hour: {peak_hour}:00 ({peak_value:.1f} W/m²)")
print(
    f"\nHours with >500 W/m² (productive hours): {len(hourly_irradiance[hourly_irradiance['mean'] > 500])} hours/day"
)
print(
    f"Hours with >800 W/m² (high-output hours): {len(hourly_irradiance[hourly_irradiance['mean'] > 800])} hours/day"
)

# Calculate daily peak sun hours
daily_energy = df.groupby("date")["irradiance_w_m2"].sum() / 1000  # Convert to kWh/m²
avg_peak_sun_hours = daily_energy.mean()
print(f"\nAverage Peak Sun Hours: {avg_peak_sun_hours:.2f} hours/day")
print("(Equivalent hours of 1,000 W/m² irradiance)")

## 4. Power Output Analysis

Analyze actual power generation from the solar PV system and compare it to theoretical maximum output. This reveals real-world system performance and efficiency losses.

In [ ]:
# Calculate theoretical maximum power output
# Theoretical power = (Irradiance / Standard Irradiance) × Nameplate Capacity
df["theoretical_power_kw"] = (df["irradiance_w_m2"] / STANDARD_IRRADIANCE) * NAMEPLATE_CAPACITY_KW
df["power_efficiency"] = (df["power_output_kw"] / df["theoretical_power_kw"]) * 100
df["power_efficiency"] = df["power_efficiency"].clip(0, 100)  # Cap at 100%

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Actual vs Theoretical Power Output
ax1 = axes[0, 0]
hourly_power = df.groupby("hour").agg({"power_output_kw": "mean", "theoretical_power_kw": "mean"})
ax1.plot(
    hourly_power.index,
    hourly_power["theoretical_power_kw"],
    linewidth=2.5,
    marker="s",
    markersize=6,
    color="#4ECDC4",
    label="Theoretical Maximum",
    linestyle="--",
)
ax1.plot(
    hourly_power.index,
    hourly_power["power_output_kw"],
    linewidth=2.5,
    marker="o",
    markersize=6,
    color="#FF6B35",
    label="Actual Output",
)
ax1.fill_between(
    hourly_power.index,
    hourly_power["power_output_kw"],
    hourly_power["theoretical_power_kw"],
    alpha=0.3,
    color="gray",
    label="Efficiency Loss",
)
ax1.axhline(
    y=NAMEPLATE_CAPACITY_KW,
    color="red",
    linestyle=":",
    linewidth=2,
    label=f"Nameplate Capacity ({NAMEPLATE_CAPACITY_KW} kW)",
)
ax1.set_xlabel("Hour of Day", fontsize=12, fontweight="bold")
ax1.set_ylabel("Power Output (kW)", fontsize=12, fontweight="bold")
ax1.set_title("Actual vs Theoretical Power Output", fontsize=14, fontweight="bold")
ax1.grid(True, alpha=0.3)
ax1.legend(loc="upper left")
ax1.set_xlim(0, 23)

# Plot 2: Power output distribution
ax2 = axes[0, 1]
ax2.hist(
    df[df["power_output_kw"] > 0]["power_output_kw"],
    bins=50,
    color="#FF6B35",
    edgecolor="black",
    alpha=0.7,
)
ax2.axvline(
    df["power_output_kw"].mean(),
    color="blue",
    linestyle="--",
    linewidth=2,
    label=f"Mean: {df['power_output_kw'].mean():.2f} kW",
)
ax2.axvline(
    df["power_output_kw"].median(),
    color="green",
    linestyle="--",
    linewidth=2,
    label=f"Median: {df['power_output_kw'].median():.2f} kW",
)
ax2.axvline(
    NAMEPLATE_CAPACITY_KW,
    color="red",
    linestyle=":",
    linewidth=2,
    label=f"Nameplate: {NAMEPLATE_CAPACITY_KW} kW",
)
ax2.set_xlabel("Power Output (kW)", fontsize=12, fontweight="bold")
ax2.set_ylabel("Frequency", fontsize=12, fontweight="bold")
ax2.set_title("Power Output Distribution", fontsize=14, fontweight="bold")
ax2.legend()
ax2.grid(True, alpha=0.3, axis="y")

# Plot 3: Seasonal power output comparison
ax3 = axes[1, 0]
seasonal_power = df.groupby("season")["power_output_kw"].agg(["mean", "std", "max"])
seasonal_order = ["Winter", "Spring", "Summer", "Fall"]
seasonal_power = seasonal_power.reindex(seasonal_order)
x_pos = np.arange(len(seasonal_order))
colors_season = ["#6C9BD1", "#98D8C8", "#FFD93D", "#F4A259"]
bars = ax3.bar(
    x_pos,
    seasonal_power["mean"],
    yerr=seasonal_power["std"],
    color=colors_season,
    edgecolor="black",
    capsize=5,
    alpha=0.8,
)
ax3.set_xlabel("Season", fontsize=12, fontweight="bold")
ax3.set_ylabel("Average Power Output (kW)", fontsize=12, fontweight="bold")
ax3.set_title("Seasonal Power Output Comparison", fontsize=14, fontweight="bold")
ax3.set_xticks(x_pos)
ax3.set_xticklabels(seasonal_order)
ax3.grid(True, alpha=0.3, axis="y")

# Add value labels
for _i, bar in enumerate(bars):
    height = bar.get_height()
    ax3.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{height:.2f} kW",
        ha="center",
        va="bottom",
        fontsize=10,
        fontweight="bold",
    )

# Plot 4: Power efficiency by irradiance level
ax4 = axes[1, 1]
# Filter data where both power and irradiance are significant
efficiency_data = df[(df["irradiance_w_m2"] > 100) & (df["power_output_kw"] > 0)].copy()
ax4.scatter(
    efficiency_data["irradiance_w_m2"],
    efficiency_data["power_efficiency"],
    alpha=0.3,
    s=10,
    color="#FF6B35",
)
# Add trend line
z = np.polyfit(efficiency_data["irradiance_w_m2"], efficiency_data["power_efficiency"], 2)
p = np.poly1d(z)
x_trend = np.linspace(
    efficiency_data["irradiance_w_m2"].min(), efficiency_data["irradiance_w_m2"].max(), 100
)
ax4.plot(x_trend, p(x_trend), "b--", linewidth=2.5, label="Trend Line")
ax4.set_xlabel("Solar Irradiance (W/m²)", fontsize=12, fontweight="bold")
ax4.set_ylabel("Power Efficiency (%)", fontsize=12, fontweight="bold")
ax4.set_title("System Efficiency vs Irradiance", fontsize=14, fontweight="bold")
ax4.grid(True, alpha=0.3)
ax4.legend()

plt.tight_layout()
plt.show()

# Power output statistics
print("=" * 60)
print("POWER OUTPUT STATISTICS")
print("=" * 60)
print(f"Maximum Recorded Power: {df['power_output_kw'].max():.2f} kW")
print(f"Average Power Output: {df['power_output_kw'].mean():.2f} kW")
print(f"Median Power Output: {df['power_output_kw'].median():.2f} kW")
print(f"\nHours Operating (>0 kW): {(df['power_output_kw'] > 0).sum():,} hours")
print(
    f"Hours at >50% Capacity: {(df['power_output_kw'] > NAMEPLATE_CAPACITY_KW * 0.5).sum():,} hours"
)
print(
    f"Hours at >80% Capacity: {(df['power_output_kw'] > NAMEPLATE_CAPACITY_KW * 0.8).sum():,} hours"
)
print(
    f"\nTheoretical vs Actual Gap: {(hourly_power['theoretical_power_kw'].mean() - hourly_power['power_output_kw'].mean()):.2f} kW"
)
print(f"Average System Efficiency: {efficiency_data['power_efficiency'].mean():.2f}%")

## 5. Temperature Effects on Performance

**Temperature coefficient** represents the rate at which solar panel efficiency decreases as temperature rises above standard test conditions (25°C). This is a critical factor in real-world performance, as panels can reach 60-70°C in hot conditions, reducing output by 15-20%.

In [ ]:
# Calculate temperature-adjusted theoretical power
df["temp_loss_factor"] = 1 + (TEMP_COEFFICIENT * (df["panel_temp_c"] - REFERENCE_TEMP_C))
df["temp_adjusted_power_kw"] = df["theoretical_power_kw"] * df["temp_loss_factor"]
df["temp_loss_kw"] = df["theoretical_power_kw"] - df["temp_adjusted_power_kw"]

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Panel temperature vs power output
ax1 = axes[0, 0]
temp_bins = df[df["power_output_kw"] > 0].copy()
ax1.scatter(
    temp_bins["panel_temp_c"],
    temp_bins["power_output_kw"],
    alpha=0.4,
    s=15,
    c=temp_bins["irradiance_w_m2"],
    cmap="YlOrRd",
)
cbar = plt.colorbar(ax1.collections[0], ax=ax1)
cbar.set_label("Irradiance (W/m²)", fontsize=11)

# Add trend line
z = np.polyfit(temp_bins["panel_temp_c"], temp_bins["power_output_kw"], 1)
p = np.poly1d(z)
x_trend = np.linspace(temp_bins["panel_temp_c"].min(), temp_bins["panel_temp_c"].max(), 100)
ax1.plot(x_trend, p(x_trend), "b--", linewidth=2.5, label="Linear Trend")

ax1.axvline(
    x=REFERENCE_TEMP_C,
    color="green",
    linestyle=":",
    linewidth=2,
    label=f"Reference Temp ({REFERENCE_TEMP_C}°C)",
)
ax1.set_xlabel("Panel Temperature (°C)", fontsize=12, fontweight="bold")
ax1.set_ylabel("Power Output (kW)", fontsize=12, fontweight="bold")
ax1.set_title("Temperature Impact on Power Output", fontsize=14, fontweight="bold")
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Temperature loss analysis
ax2 = axes[0, 1]
temp_range_bins = pd.cut(df["panel_temp_c"], bins=10)
temp_loss_by_range = df.groupby(temp_range_bins)["temp_loss_kw"].mean()
temp_midpoints = [interval.mid for interval in temp_loss_by_range.index]
colors_gradient = plt.cm.RdYlBu_r(np.linspace(0.2, 0.9, len(temp_midpoints)))
bars = ax2.bar(
    range(len(temp_midpoints)), temp_loss_by_range.values, color=colors_gradient, edgecolor="black"
)
ax2.set_xlabel("Panel Temperature Range (°C)", fontsize=12, fontweight="bold")
ax2.set_ylabel("Average Power Loss (kW)", fontsize=12, fontweight="bold")
ax2.set_title("Temperature-Induced Power Loss", fontsize=14, fontweight="bold")
ax2.set_xticks(range(len(temp_midpoints)))
ax2.set_xticklabels([f"{x:.0f}" for x in temp_midpoints], rotation=45)
ax2.grid(True, alpha=0.3, axis="y")

# Add value labels
for _i, bar in enumerate(bars):
    height = bar.get_height()
    if not np.isnan(height):
        ax2.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{height:.2f}",
            ha="center",
            va="bottom",
            fontsize=9,
        )

# Plot 3: Daily temperature patterns
ax3 = axes[1, 0]
hourly_temp = df.groupby("hour").agg({"ambient_temp_c": "mean", "panel_temp_c": "mean"})
ax3.plot(
    hourly_temp.index,
    hourly_temp["ambient_temp_c"],
    linewidth=2.5,
    marker="o",
    markersize=6,
    color="#4ECDC4",
    label="Ambient Temperature",
)
ax3.plot(
    hourly_temp.index,
    hourly_temp["panel_temp_c"],
    linewidth=2.5,
    marker="s",
    markersize=6,
    color="#FF6B35",
    label="Panel Temperature",
)
ax3.fill_between(
    hourly_temp.index,
    hourly_temp["ambient_temp_c"],
    hourly_temp["panel_temp_c"],
    alpha=0.3,
    color="orange",
    label="Temperature Rise",
)
ax3.axhline(
    y=REFERENCE_TEMP_C,
    color="green",
    linestyle="--",
    linewidth=2,
    label=f"Reference Temp ({REFERENCE_TEMP_C}°C)",
)
ax3.set_xlabel("Hour of Day", fontsize=12, fontweight="bold")
ax3.set_ylabel("Temperature (°C)", fontsize=12, fontweight="bold")
ax3.set_title("Daily Temperature Patterns", fontsize=14, fontweight="bold")
ax3.grid(True, alpha=0.3)
ax3.legend()
ax3.set_xlim(0, 23)

# Plot 4: Efficiency vs temperature
ax4 = axes[1, 1]
efficiency_temp = df[(df["power_output_kw"] > 5) & (df["irradiance_w_m2"] > 500)].copy()
efficiency_temp["efficiency_pct"] = (
    efficiency_temp["power_output_kw"] / efficiency_temp["theoretical_power_kw"]
) * 100
temp_bins_plot = pd.cut(efficiency_temp["panel_temp_c"], bins=15)
efficiency_by_temp = efficiency_temp.groupby(temp_bins_plot)["efficiency_pct"].mean()
temp_midpoints_eff = [interval.mid for interval in efficiency_by_temp.index]
ax4.plot(
    temp_midpoints_eff,
    efficiency_by_temp.values,
    linewidth=3,
    marker="o",
    markersize=8,
    color="#FF6B35",
)

# Add theoretical efficiency line
theoretical_eff = 100 * (1 + TEMP_COEFFICIENT * (np.array(temp_midpoints_eff) - REFERENCE_TEMP_C))
ax4.plot(
    temp_midpoints_eff,
    theoretical_eff,
    linewidth=2.5,
    linestyle="--",
    color="blue",
    label="Theoretical",
)

ax4.axvline(
    x=REFERENCE_TEMP_C,
    color="green",
    linestyle=":",
    linewidth=2,
    label=f"Ref Temp ({REFERENCE_TEMP_C}°C)",
)
ax4.set_xlabel("Panel Temperature (°C)", fontsize=12, fontweight="bold")
ax4.set_ylabel("System Efficiency (%)", fontsize=12, fontweight="bold")
ax4.set_title("Efficiency vs Panel Temperature", fontsize=14, fontweight="bold")
ax4.grid(True, alpha=0.3)
ax4.legend()

plt.tight_layout()
plt.show()

# Temperature impact statistics
print("=" * 60)
print("TEMPERATURE IMPACT ANALYSIS")
print("=" * 60)
print(f"Average Ambient Temperature: {df['ambient_temp_c'].mean():.2f}°C")
print(f"Average Panel Temperature: {df['panel_temp_c'].mean():.2f}°C")
print(f"Maximum Panel Temperature: {df['panel_temp_c'].max():.2f}°C")
print(
    f"\nAverage Panel Temperature Rise: {(df['panel_temp_c'] - df['ambient_temp_c']).mean():.2f}°C"
)
print(f"\nAverage Temperature Loss Factor: {(1 - df['temp_loss_factor'].mean()) * 100:.2f}%")
print(f"Total Energy Lost to Temperature: {df['temp_loss_kw'].sum():.2f} kWh/year")
print(f"Average Power Loss per Hour: {df[df['power_output_kw'] > 0]['temp_loss_kw'].mean():.2f} kW")
print(
    f"\nHours Above Reference Temp (25°C): {(df['panel_temp_c'] > REFERENCE_TEMP_C).sum():,} ({(df['panel_temp_c'] > REFERENCE_TEMP_C).sum() / len(df) * 100:.1f}%)"
)
print(
    f"Hours Above 50°C: {(df['panel_temp_c'] > 50).sum():,} ({(df['panel_temp_c'] > 50).sum() / len(df) * 100:.1f}%)"
)

## 6. Performance Metrics

Calculate industry-standard solar PV performance metrics:

- **Capacity Factor**: Ratio of actual energy output to theoretical maximum if system ran at nameplate capacity 24/7
- **Energy Yield**: Total electricity generated over the year (kWh/year)
- **Specific Yield**: Energy generated per kW of installed capacity (kWh/kWp/year)
- **Performance Ratio (PR)**: Actual output divided by expected output under ideal conditions (accounts for all losses)

In [ ]:
# Calculate key performance metrics

# 1. Energy Yield (kWh/year)
total_energy_yield_kwh = df["power_output_kw"].sum()
theoretical_max_energy_kwh = NAMEPLATE_CAPACITY_KW * 8760  # kW × hours/year

# 2. Capacity Factor (%)
capacity_factor = (total_energy_yield_kwh / theoretical_max_energy_kwh) * 100

# 3. Specific Yield (kWh/kWp/year)
specific_yield = total_energy_yield_kwh / NAMEPLATE_CAPACITY_KW

# 4. Performance Ratio (PR)
# PR = Actual Energy / (Reference Energy × (Actual Irradiation / Reference Irradiation))
reference_energy = df["theoretical_power_kw"].sum()
performance_ratio = (total_energy_yield_kwh / reference_energy) * 100 if reference_energy > 0 else 0

# 5. Monthly performance
monthly_metrics = (
    df.groupby("month")
    .agg({"power_output_kw": "sum", "theoretical_power_kw": "sum", "irradiance_w_m2": "mean"})
    .reset_index()
)
monthly_metrics["capacity_factor"] = (
    monthly_metrics["power_output_kw"] / (NAMEPLATE_CAPACITY_KW * 730)
) * 100  # Approx 730 hrs/month
monthly_metrics["performance_ratio"] = (
    monthly_metrics["power_output_kw"] / monthly_metrics["theoretical_power_kw"]
) * 100

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Monthly energy production
ax1 = axes[0, 0]
month_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
colors_gradient = plt.cm.viridis(np.linspace(0.3, 0.9, 12))
bars = ax1.bar(
    monthly_metrics["month"],
    monthly_metrics["power_output_kw"],
    color=colors_gradient,
    edgecolor="black",
    linewidth=1.5,
)
ax1.set_xlabel("Month", fontsize=12, fontweight="bold")
ax1.set_ylabel("Energy Production (kWh)", fontsize=12, fontweight="bold")
ax1.set_title("Monthly Energy Production", fontsize=14, fontweight="bold")
ax1.set_xticks(monthly_metrics["month"])
ax1.set_xticklabels(month_labels)
ax1.grid(True, alpha=0.3, axis="y")

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax1.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{height:.0f}",
        ha="center",
        va="bottom",
        fontsize=9,
    )

# Plot 2: Monthly capacity factor
ax2 = axes[0, 1]
ax2.plot(
    monthly_metrics["month"],
    monthly_metrics["capacity_factor"],
    linewidth=3,
    marker="o",
    markersize=10,
    color="#FF6B35",
)
ax2.axhline(
    y=capacity_factor,
    color="blue",
    linestyle="--",
    linewidth=2,
    label=f"Annual Avg: {capacity_factor:.2f}%",
)
ax2.fill_between(
    monthly_metrics["month"], 0, monthly_metrics["capacity_factor"], alpha=0.3, color="#FF6B35"
)
ax2.set_xlabel("Month", fontsize=12, fontweight="bold")
ax2.set_ylabel("Capacity Factor (%)", fontsize=12, fontweight="bold")
ax2.set_title("Monthly Capacity Factor", fontsize=14, fontweight="bold")
ax2.set_xticks(monthly_metrics["month"])
ax2.set_xticklabels(month_labels)
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_ylim(0, None)

# Plot 3: Monthly performance ratio
ax3 = axes[1, 0]
colors_pr = [
    "#27AE60" if pr >= 75 else "#F39C12" if pr >= 70 else "#E74C3C"
    for pr in monthly_metrics["performance_ratio"]
]
bars = ax3.bar(
    monthly_metrics["month"],
    monthly_metrics["performance_ratio"],
    color=colors_pr,
    edgecolor="black",
    linewidth=1.5,
)
ax3.axhline(
    y=performance_ratio,
    color="blue",
    linestyle="--",
    linewidth=2,
    label=f"Annual PR: {performance_ratio:.2f}%",
)
ax3.axhline(y=75, color="green", linestyle=":", linewidth=2, alpha=0.5, label="Target PR: 75%")
ax3.set_xlabel("Month", fontsize=12, fontweight="bold")
ax3.set_ylabel("Performance Ratio (%)", fontsize=12, fontweight="bold")
ax3.set_title("Monthly Performance Ratio", fontsize=14, fontweight="bold")
ax3.set_xticks(monthly_metrics["month"])
ax3.set_xticklabels(month_labels)
ax3.grid(True, alpha=0.3, axis="y")
ax3.legend()

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax3.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{height:.1f}%",
        ha="center",
        va="bottom",
        fontsize=9,
    )

# Plot 4: Key metrics dashboard
ax4 = axes[1, 1]
ax4.axis("off")

# Create metrics text
metrics_text = f"""
ANNUAL PERFORMANCE METRICS
{"=" * 50}

System Configuration:
  • Nameplate Capacity: {NAMEPLATE_CAPACITY_KW} kW
  • Base Efficiency: {BASE_EFFICIENCY * 100}%
  • Temp Coefficient: {TEMP_COEFFICIENT * 100}%/°C

Energy Production:
  • Total Energy Yield: {total_energy_yield_kwh:,.0f} kWh/year
  • Specific Yield: {specific_yield:.0f} kWh/kWp/year
  • Average Daily Production: {total_energy_yield_kwh / 365:.1f} kWh/day

Performance Indicators:
  • Capacity Factor: {capacity_factor:.2f}%
  • Performance Ratio: {performance_ratio:.2f}%
  • System Availability: {(df["power_output_kw"] > 0).sum() / len(df) * 100:.1f}%

Benchmarking:
  • Industry Avg CF: 15-25% (This system: {capacity_factor:.1f}%)
  • Target PR: >75% (This system: {performance_ratio:.1f}%)
  • System Status: {"✓ Excellent" if performance_ratio >= 80 else "✓ Good" if performance_ratio >= 75 else "⚠ Needs Attention"}

Peak Performance:
  • Best Month: {month_labels[monthly_metrics["power_output_kw"].idxmax() - 1]} ({monthly_metrics["power_output_kw"].max():.0f} kWh)
  • Worst Month: {month_labels[monthly_metrics["power_output_kw"].idxmin() - 1]} ({monthly_metrics["power_output_kw"].min():.0f} kWh)
  • Seasonal Variation: {(monthly_metrics["power_output_kw"].max() / monthly_metrics["power_output_kw"].min()):.1f}x
"""

ax4.text(
    0.1,
    0.95,
    metrics_text,
    transform=ax4.transAxes,
    fontsize=11,
    verticalalignment="top",
    fontfamily="monospace",
    bbox={"boxstyle": "round", "facecolor": "wheat", "alpha": 0.3},
)

plt.tight_layout()
plt.show()

# Print detailed metrics
print("=" * 60)
print("COMPREHENSIVE PERFORMANCE METRICS")
print("=" * 60)
print("\n1. ENERGY YIELD")
print(f"   Total Annual Production: {total_energy_yield_kwh:,.2f} kWh")
print(f"   Specific Yield: {specific_yield:.2f} kWh/kWp/year")
print(f"   Average Daily: {total_energy_yield_kwh / 365:.2f} kWh")
print(f"   Average Hourly: {df['power_output_kw'].mean():.2f} kW")

print("\n2. CAPACITY FACTOR")
print(f"   Annual Capacity Factor: {capacity_factor:.2f}%")
print(f"   Theoretical Maximum: {theoretical_max_energy_kwh:,.0f} kWh")
print(f"   Actual Production: {total_energy_yield_kwh:,.0f} kWh")
print(f"   Utilization Gap: {theoretical_max_energy_kwh - total_energy_yield_kwh:,.0f} kWh")

print("\n3. PERFORMANCE RATIO")
print(f"   System PR: {performance_ratio:.2f}%")
print("   Industry Benchmark: 75-85%")
print(f"   Status: {'Above Benchmark' if performance_ratio >= 75 else 'Below Benchmark'}")

print("\n4. OPERATIONAL STATISTICS")
print(f"   Total Operating Hours: {(df['power_output_kw'] > 0).sum():,}")
print(f"   Average Operating Hours/Day: {(df['power_output_kw'] > 0).sum() / 365:.1f}")
print(f"   System Availability: {(df['power_output_kw'] > 0).sum() / len(df) * 100:.2f}%")
print(
    f"   Peak Output Achieved: {df['power_output_kw'].max():.2f} kW ({df['power_output_kw'].max() / NAMEPLATE_CAPACITY_KW * 100:.1f}% of nameplate)"
)

## 7. Economic Analysis

Evaluate the financial performance and return on investment for the solar PV system:

- **Revenue**: Income from electricity sales based on time-varying prices
- **LCOE** (Levelized Cost of Energy): Average cost per kWh over system lifetime
- **Payback Period**: Time required to recover initial investment
- **Value Optimization**: Maximizing revenue through time-of-use pricing

In [ ]:
# Calculate economic metrics

# Revenue from electricity sales
df["revenue"] = df["power_output_kw"] * df["electricity_price_kwh"]
total_annual_revenue = df["revenue"].sum()

# Economic assumptions
SYSTEM_COST = 75000  # $1,500/kW × 50 kW (typical installation cost)
ANNUAL_OM_COST = 1000  # Annual operations & maintenance
SYSTEM_LIFETIME_YEARS = 25
DISCOUNT_RATE = 0.05  # 5% discount rate
DEGRADATION_RATE = 0.005  # 0.5% annual degradation

# Calculate LCOE (simplified)
# LCOE = (Total Lifetime Costs) / (Total Lifetime Energy Production)
total_lifetime_costs = SYSTEM_COST + (ANNUAL_OM_COST * SYSTEM_LIFETIME_YEARS)
total_lifetime_energy = sum(
    [
        total_energy_yield_kwh * (1 - DEGRADATION_RATE) ** year
        for year in range(SYSTEM_LIFETIME_YEARS)
    ]
)
lcoe = total_lifetime_costs / total_lifetime_energy

# Simple payback period
net_annual_cash_flow = total_annual_revenue - ANNUAL_OM_COST
simple_payback_years = (
    SYSTEM_COST / net_annual_cash_flow if net_annual_cash_flow > 0 else float("inf")
)

# Monthly economics
monthly_economics = (
    df.groupby("month")
    .agg({"power_output_kw": "sum", "revenue": "sum", "electricity_price_kwh": "mean"})
    .reset_index()
)
monthly_economics["avg_price_per_kwh"] = (
    monthly_economics["revenue"] / monthly_economics["power_output_kw"]
)

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Monthly revenue
ax1 = axes[0, 0]
month_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
colors_revenue = plt.cm.Greens(np.linspace(0.4, 0.9, 12))
bars = ax1.bar(
    monthly_economics["month"],
    monthly_economics["revenue"],
    color=colors_revenue,
    edgecolor="black",
    linewidth=1.5,
)
ax1.set_xlabel("Month", fontsize=12, fontweight="bold")
ax1.set_ylabel("Revenue ($)", fontsize=12, fontweight="bold")
ax1.set_title("Monthly Revenue from Electricity Sales", fontsize=14, fontweight="bold")
ax1.set_xticks(monthly_economics["month"])
ax1.set_xticklabels(month_labels)
ax1.grid(True, alpha=0.3, axis="y")

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax1.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"${height:.0f}",
        ha="center",
        va="bottom",
        fontsize=9,
    )

# Plot 2: Revenue vs energy production
ax2 = axes[0, 1]
ax2_twin = ax2.twinx()
x_pos = np.arange(len(month_labels))
width = 0.35
bars1 = ax2.bar(
    x_pos - width / 2,
    monthly_economics["power_output_kw"],
    width,
    label="Energy (kWh)",
    color="#3498DB",
    edgecolor="black",
)
bars2 = ax2_twin.bar(
    x_pos + width / 2,
    monthly_economics["revenue"],
    width,
    label="Revenue ($)",
    color="#27AE60",
    edgecolor="black",
)
ax2.set_xlabel("Month", fontsize=12, fontweight="bold")
ax2.set_ylabel("Energy Production (kWh)", fontsize=12, fontweight="bold", color="#3498DB")
ax2_twin.set_ylabel("Revenue ($)", fontsize=12, fontweight="bold", color="#27AE60")
ax2.set_title("Energy Production vs Revenue", fontsize=14, fontweight="bold")
ax2.set_xticks(x_pos)
ax2.set_xticklabels(month_labels)
ax2.tick_params(axis="y", labelcolor="#3498DB")
ax2_twin.tick_params(axis="y", labelcolor="#27AE60")
ax2.grid(True, alpha=0.3, axis="y")

# Combine legends
lines1, labels1 = ax2.get_legend_handles_labels()
lines2, labels2 = ax2_twin.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc="upper left")

# Plot 3: Cumulative cash flow (payback visualization)
ax3 = axes[1, 0]
years = np.arange(0, min(26, int(simple_payback_years) + 5))
cumulative_revenue = [net_annual_cash_flow * year for year in years]
cumulative_cash_flow = [-SYSTEM_COST + net_annual_cash_flow * year for year in years]
ax3.plot(
    years,
    cumulative_cash_flow,
    linewidth=3,
    marker="o",
    markersize=7,
    color="#27AE60",
    label="Cumulative Cash Flow",
)
ax3.axhline(y=0, color="red", linestyle="--", linewidth=2, label="Break-even")
ax3.fill_between(
    years,
    cumulative_cash_flow,
    0,
    where=np.array(cumulative_cash_flow) < 0,
    alpha=0.3,
    color="red",
    label="Payback Period",
)
ax3.fill_between(
    years,
    cumulative_cash_flow,
    0,
    where=np.array(cumulative_cash_flow) >= 0,
    alpha=0.3,
    color="green",
    label="Profit Period",
)
if simple_payback_years < 25:
    ax3.axvline(
        x=simple_payback_years,
        color="blue",
        linestyle=":",
        linewidth=2,
        label=f"Payback: {simple_payback_years:.1f} years",
    )
ax3.set_xlabel("Years", fontsize=12, fontweight="bold")
ax3.set_ylabel("Cumulative Cash Flow ($)", fontsize=12, fontweight="bold")
ax3.set_title("Investment Payback Period", fontsize=14, fontweight="bold")
ax3.grid(True, alpha=0.3)
ax3.legend()

# Plot 4: Economic metrics summary
ax4 = axes[1, 1]
ax4.axis("off")

# Calculate additional metrics
lifetime_revenue = sum(
    [total_annual_revenue * (1 - DEGRADATION_RATE) ** year for year in range(SYSTEM_LIFETIME_YEARS)]
)
lifetime_om = ANNUAL_OM_COST * SYSTEM_LIFETIME_YEARS
lifetime_profit = lifetime_revenue - SYSTEM_COST - lifetime_om
roi = (lifetime_profit / SYSTEM_COST) * 100

economic_text = f"""
ECONOMIC ANALYSIS SUMMARY
{"=" * 50}

Investment Details:
  • System Cost: ${SYSTEM_COST:,.0f}
  • Cost per Watt: ${SYSTEM_COST / (NAMEPLATE_CAPACITY_KW * 1000):.2f}/W
  • Annual O&M: ${ANNUAL_OM_COST:,.0f}/year
  • System Lifetime: {SYSTEM_LIFETIME_YEARS} years

Annual Performance:
  • Energy Production: {total_energy_yield_kwh:,.0f} kWh/year
  • Gross Revenue: ${total_annual_revenue:,.2f}/year
  • Net Revenue: ${net_annual_cash_flow:,.2f}/year
  • Avg Price Received: ${total_annual_revenue / total_energy_yield_kwh:.3f}/kWh

Key Financial Metrics:
  • LCOE: ${lcoe:.3f}/kWh
  • Simple Payback: {simple_payback_years:.1f} years
  • NPV (25 years): ${lifetime_profit:,.0f}
  • ROI: {roi:.1f}%

Lifetime Economics (25 years):
  • Total Energy: {total_lifetime_energy:,.0f} kWh
  • Total Revenue: ${lifetime_revenue:,.0f}
  • Total Costs: ${SYSTEM_COST + lifetime_om:,.0f}
  • Net Profit: ${lifetime_profit:,.0f}

Economic Viability:
  • Status: {"✓ Excellent Investment" if roi > 200 else "✓ Good Investment" if roi > 100 else "⚠ Marginal"}
  • Payback Status: {"✓ Fast Payback" if simple_payback_years < 8 else "✓ Acceptable" if simple_payback_years < 12 else "⚠ Slow"}
"""

ax4.text(
    0.1,
    0.95,
    economic_text,
    transform=ax4.transAxes,
    fontsize=11,
    verticalalignment="top",
    fontfamily="monospace",
    bbox={"boxstyle": "round", "facecolor": "lightgreen", "alpha": 0.3},
)

plt.tight_layout()
plt.show()

print("=" * 60)
print("DETAILED ECONOMIC METRICS")
print("=" * 60)
print("\n1. REVENUE ANALYSIS")
print(f"   Total Annual Revenue: ${total_annual_revenue:,.2f}")
print(f"   Average Monthly Revenue: ${total_annual_revenue / 12:,.2f}")
print(f"   Revenue per kWh: ${total_annual_revenue / total_energy_yield_kwh:.3f}")
print(
    f"   Best Month: {month_labels[monthly_economics['revenue'].idxmax() - 1]} (${monthly_economics['revenue'].max():.2f})"
)
print(
    f"   Worst Month: {month_labels[monthly_economics['revenue'].idxmin() - 1]} (${monthly_economics['revenue'].min():.2f})"
)

print("\n2. COST METRICS")
print(f"   LCOE: ${lcoe:.4f}/kWh")
print(f"   Grid Electricity Avg: ${df['electricity_price_kwh'].mean():.3f}/kWh")
print(f"   Savings vs Grid: ${df['electricity_price_kwh'].mean() - lcoe:.4f}/kWh")

print("\n3. RETURN ON INVESTMENT")
print(f"   Simple Payback Period: {simple_payback_years:.2f} years")
print(f"   25-Year ROI: {roi:.2f}%")
print(f"   Annual ROI: {roi / SYSTEM_LIFETIME_YEARS:.2f}%")
print(f"   Lifetime Profit: ${lifetime_profit:,.2f}")

## 8. Time-of-Use Optimization

**Time-of-Use (TOU)** pricing means electricity prices vary by time of day, typically higher during peak demand hours. Solar generation naturally aligns with daytime peaks, creating additional economic value.

In [ ]:
# Analyze time-of-use pricing patterns and optimization opportunities

# Define time-of-use periods (typical utility structure)
def classify_tou_period(hour):
    if 9 <= hour <= 20:  # Peak hours
        return "Peak"
    elif 6 <= hour < 9 or 21 <= hour < 23:  # Shoulder hours
        return "Shoulder"
    else:  # Off-peak hours
        return "Off-Peak"


df["tou_period"] = df["hour"].apply(classify_tou_period)

# Calculate value by TOU period
tou_analysis = (
    df.groupby("tou_period")
    .agg({"power_output_kw": ["sum", "mean"], "electricity_price_kwh": "mean", "revenue": "sum"})
    .reset_index()
)
tou_analysis.columns = ["tou_period", "total_energy", "avg_power", "avg_price", "total_revenue"]
tou_analysis["energy_pct"] = (
    tou_analysis["total_energy"] / tou_analysis["total_energy"].sum()
) * 100
tou_analysis["revenue_pct"] = (
    tou_analysis["total_revenue"] / tou_analysis["total_revenue"].sum()
) * 100

# Hourly analysis
hourly_tou = (
    df.groupby("hour")
    .agg({"power_output_kw": "mean", "electricity_price_kwh": "mean", "revenue": "mean"})
    .reset_index()
)
hourly_tou["value_per_kwh"] = hourly_tou["revenue"] / hourly_tou["power_output_kw"]

fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: Price vs generation alignment
ax1 = axes[0, 0]
ax1_twin = ax1.twinx()
line1 = ax1.plot(
    hourly_tou["hour"],
    hourly_tou["power_output_kw"],
    linewidth=3,
    marker="o",
    markersize=7,
    color="#3498DB",
    label="Solar Generation",
)
line2 = ax1_twin.plot(
    hourly_tou["hour"],
    hourly_tou["electricity_price_kwh"],
    linewidth=3,
    marker="s",
    markersize=7,
    color="#E74C3C",
    label="Electricity Price",
)

# Shade TOU periods
ax1.axvspan(0, 6, alpha=0.2, color="blue", label="Off-Peak")
ax1.axvspan(6, 9, alpha=0.2, color="yellow", label="Shoulder")
ax1.axvspan(9, 20, alpha=0.2, color="red", label="Peak")
ax1.axvspan(21, 23, alpha=0.2, color="yellow")
ax1.axvspan(23, 24, alpha=0.2, color="blue")

ax1.set_xlabel("Hour of Day", fontsize=12, fontweight="bold")
ax1.set_ylabel("Solar Generation (kW)", fontsize=12, fontweight="bold", color="#3498DB")
ax1_twin.set_ylabel("Electricity Price ($/kWh)", fontsize=12, fontweight="bold", color="#E74C3C")
ax1.set_title("Solar Generation vs Electricity Price Alignment", fontsize=14, fontweight="bold")
ax1.tick_params(axis="y", labelcolor="#3498DB")
ax1_twin.tick_params(axis="y", labelcolor="#E74C3C")
ax1.set_xlim(0, 23)
ax1.grid(True, alpha=0.3)

# Combine legends
lines = line1 + line2
labels = [line.get_label() for line in lines]
ax1.legend(lines, labels, loc="upper left")

# Plot 2: TOU period energy and revenue distribution
ax2 = axes[0, 1]
tou_order = ["Off-Peak", "Shoulder", "Peak"]
tou_analysis_sorted = tou_analysis.set_index("tou_period").reindex(tou_order).reset_index()
x_pos = np.arange(len(tou_order))
width = 0.35
bars1 = ax2.bar(
    x_pos - width / 2,
    tou_analysis_sorted["energy_pct"],
    width,
    label="Energy %",
    color="#3498DB",
    edgecolor="black",
)
bars2 = ax2.bar(
    x_pos + width / 2,
    tou_analysis_sorted["revenue_pct"],
    width,
    label="Revenue %",
    color="#27AE60",
    edgecolor="black",
)
ax2.set_xlabel("Time-of-Use Period", fontsize=12, fontweight="bold")
ax2.set_ylabel("Percentage (%)", fontsize=12, fontweight="bold")
ax2.set_title("Energy vs Revenue Distribution by TOU Period", fontsize=14, fontweight="bold")
ax2.set_xticks(x_pos)
ax2.set_xticklabels(tou_order)
ax2.legend()
ax2.grid(True, alpha=0.3, axis="y")

# Add value labels
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax2.text(
            bar.get_x() + bar.get_width() / 2.0,
            height,
            f"{height:.1f}%",
            ha="center",
            va="bottom",
            fontsize=9,
        )

# Plot 3: Hourly revenue optimization
ax3 = axes[1, 0]
hourly_revenue = df.groupby("hour")["revenue"].mean()
colors_hourly = [
    "#2C3E50" if h < 6 or h >= 23 else "#F39C12" if 6 <= h < 9 or 21 <= h < 23 else "#E74C3C"
    for h in hourly_revenue.index
]
bars = ax3.bar(
    hourly_revenue.index, hourly_revenue.values, color=colors_hourly, edgecolor="black", linewidth=1
)
ax3.set_xlabel("Hour of Day", fontsize=12, fontweight="bold")
ax3.set_ylabel("Average Revenue ($/hour)", fontsize=12, fontweight="bold")
ax3.set_title("Hourly Revenue Profile", fontsize=14, fontweight="bold")
ax3.grid(True, alpha=0.3, axis="y")

# Add legend for colors
from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor="#2C3E50", label="Off-Peak"),
    Patch(facecolor="#F39C12", label="Shoulder"),
    Patch(facecolor="#E74C3C", label="Peak"),
]
ax3.legend(handles=legend_elements, loc="upper left")

# Plot 4: Value optimization summary
ax4 = axes[1, 1]
ax4.axis("off")

# Calculate optimization metrics
peak_alignment = tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Peak"][
    "energy_pct"
].values[0]
peak_revenue = tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Peak"][
    "total_revenue"
].values[0]
avg_peak_price = tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Peak"][
    "avg_price"
].values[0]
avg_offpeak_price = tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Off-Peak"][
    "avg_price"
].values[0]
price_premium = ((avg_peak_price - avg_offpeak_price) / avg_offpeak_price) * 100

# Calculate flat rate comparison
flat_rate_revenue = total_energy_yield_kwh * df["electricity_price_kwh"].mean()
tou_advantage = total_annual_revenue - flat_rate_revenue
tou_advantage_pct = (tou_advantage / flat_rate_revenue) * 100

optimization_text = f"""
TIME-OF-USE OPTIMIZATION ANALYSIS
{"=" * 50}

TOU Period Breakdown:
  • Off-Peak (23:00-06:00):
    - Energy: {tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Off-Peak"]["energy_pct"].values[0]:.1f}%
    - Revenue: {tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Off-Peak"]["revenue_pct"].values[0]:.1f}%
    - Avg Price: ${tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Off-Peak"]["avg_price"].values[0]:.3f}/kWh

  • Shoulder (06:00-09:00, 21:00-23:00):
    - Energy: {tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Shoulder"]["energy_pct"].values[0]:.1f}%
    - Revenue: {tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Shoulder"]["revenue_pct"].values[0]:.1f}%
    - Avg Price: ${tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Shoulder"]["avg_price"].values[0]:.3f}/kWh

  • Peak (09:00-20:00):
    - Energy: {peak_alignment:.1f}%
    - Revenue: {tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Peak"]["revenue_pct"].values[0]:.1f}%
    - Avg Price: ${avg_peak_price:.3f}/kWh

Value Optimization:
  • Peak/Off-Peak Price Ratio: {price_premium:.1f}% premium
  • Solar-Peak Alignment: {peak_alignment:.1f}% of energy in peak hours
  • Peak Period Revenue: ${peak_revenue:.2f} ({tou_analysis_sorted[tou_analysis_sorted["tou_period"] == "Peak"]["revenue_pct"].values[0]:.1f}% of total)

TOU vs Flat Rate:
  • Flat Rate Revenue: ${flat_rate_revenue:.2f}
  • TOU Revenue: ${total_annual_revenue:.2f}
  • Additional Value: ${tou_advantage:.2f} ({tou_advantage_pct:+.1f}%)

Optimization Status:
  • Alignment Score: {"✓ Excellent" if peak_alignment > 70 else "✓ Good" if peak_alignment > 60 else "⚠ Fair"}
  • Peak Capture: {peak_alignment:.1f}% (Target: >70%)
  • Value Premium: {"✓ Strong" if tou_advantage_pct > 5 else "⚠ Moderate"}
"""

ax4.text(
    0.1,
    0.95,
    optimization_text,
    transform=ax4.transAxes,
    fontsize=11,
    verticalalignment="top",
    fontfamily="monospace",
    bbox={"boxstyle": "round", "facecolor": "lightyellow", "alpha": 0.3},
)

plt.tight_layout()
plt.show()

print("=" * 60)
print("TIME-OF-USE OPTIMIZATION INSIGHTS")
print("=" * 60)
print("\nSolar generation naturally aligns with peak pricing hours!")
print("\nPeak Hour Performance:")
print(f"  • {peak_alignment:.1f}% of total energy generated during peak hours")
print(
    f"  • Peak hours capture {tou_analysis_sorted[tou_analysis_sorted['tou_period'] == 'Peak']['revenue_pct'].values[0]:.1f}% of total revenue"
)
print(f"  • Peak price premium: {price_premium:.1f}% above off-peak rates")
print("\nEconomic Advantage:")
print(f"  • TOU pricing adds ${tou_advantage:.2f} annual value ({tou_advantage_pct:+.1f}%)")
print(f"  • Average realized price: ${total_annual_revenue / total_energy_yield_kwh:.3f}/kWh")
print(f"  • vs. flat average rate: ${df['electricity_price_kwh'].mean():.3f}/kWh")


## 9. Seasonal Comparison

Compare system performance across seasons to understand variation in solar resource availability and efficiency.

In [ ]:
# Comprehensive seasonal analysis

seasonal_metrics = (
    df.groupby("season")
    .agg(
        {
            "irradiance_w_m2": ["mean", "max"],
            "power_output_kw": ["sum", "mean", "max"],
            "panel_temp_c": ["mean", "max"],
            "ambient_temp_c": ["mean", "max"],
            "revenue": "sum",
            "power_efficiency": "mean",
        }
    )
    .reset_index()
)

seasonal_metrics.columns = [
    "season",
    "avg_irradiance",
    "max_irradiance",
    "total_energy",
    "avg_power",
    "max_power",
    "avg_panel_temp",
    "max_panel_temp",
    "avg_ambient_temp",
    "max_ambient_temp",
    "total_revenue",
    "avg_efficiency",
]

# Calculate seasonal capacity factors
hours_per_season = df.groupby("season").size()
seasonal_metrics["capacity_factor"] = (
    seasonal_metrics["total_energy"] / (NAMEPLATE_CAPACITY_KW * hours_per_season.values)
) * 100

# Reorder by calendar order
season_order = ["Winter", "Spring", "Summer", "Fall"]
seasonal_metrics = seasonal_metrics.set_index("season").reindex(season_order).reset_index()

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot 1: Seasonal energy production
ax1 = axes[0, 0]
colors_season = ["#6C9BD1", "#98D8C8", "#FFD93D", "#F4A259"]
bars = ax1.bar(
    seasonal_metrics["season"],
    seasonal_metrics["total_energy"],
    color=colors_season,
    edgecolor="black",
    linewidth=2,
)
ax1.set_ylabel("Total Energy (kWh)", fontsize=12, fontweight="bold")
ax1.set_title("Seasonal Energy Production", fontsize=13, fontweight="bold")
ax1.grid(True, alpha=0.3, axis="y")
for _i, bar in enumerate(bars):
    height = bar.get_height()
    ax1.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{height:.0f}\nkWh",
        ha="center",
        va="bottom",
        fontsize=10,
        fontweight="bold",
    )

# Plot 2: Seasonal capacity factor
ax2 = axes[0, 1]
bars = ax2.bar(
    seasonal_metrics["season"],
    seasonal_metrics["capacity_factor"],
    color=colors_season,
    edgecolor="black",
    linewidth=2,
)
ax2.axhline(
    y=capacity_factor,
    color="red",
    linestyle="--",
    linewidth=2,
    label=f"Annual Avg: {capacity_factor:.1f}%",
)
ax2.set_ylabel("Capacity Factor (%)", fontsize=12, fontweight="bold")
ax2.set_title("Seasonal Capacity Factor", fontsize=13, fontweight="bold")
ax2.grid(True, alpha=0.3, axis="y")
ax2.legend()
for _i, bar in enumerate(bars):
    height = bar.get_height()
    ax2.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{height:.1f}%",
        ha="center",
        va="bottom",
        fontsize=10,
        fontweight="bold",
    )

# Plot 3: Seasonal irradiance
ax3 = axes[0, 2]
bars = ax3.bar(
    seasonal_metrics["season"],
    seasonal_metrics["avg_irradiance"],
    color=colors_season,
    edgecolor="black",
    linewidth=2,
)
ax3.set_ylabel("Avg Irradiance (W/m²)", fontsize=12, fontweight="bold")
ax3.set_title("Seasonal Average Irradiance", fontsize=13, fontweight="bold")
ax3.grid(True, alpha=0.3, axis="y")
for _i, bar in enumerate(bars):
    height = bar.get_height()
    ax3.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"{height:.0f}",
        ha="center",
        va="bottom",
        fontsize=10,
        fontweight="bold",
    )

# Plot 4: Temperature comparison
ax4 = axes[1, 0]
x_pos = np.arange(len(season_order))
width = 0.35
bars1 = ax4.bar(
    x_pos - width / 2,
    seasonal_metrics["avg_ambient_temp"],
    width,
    label="Ambient",
    color="#3498DB",
    edgecolor="black",
)
bars2 = ax4.bar(
    x_pos + width / 2,
    seasonal_metrics["avg_panel_temp"],
    width,
    label="Panel",
    color="#E74C3C",
    edgecolor="black",
)
ax4.axhline(
    y=REFERENCE_TEMP_C,
    color="green",
    linestyle="--",
    linewidth=2,
    label=f"Reference ({REFERENCE_TEMP_C}°C)",
)
ax4.set_ylabel("Temperature (°C)", fontsize=12, fontweight="bold")
ax4.set_title("Seasonal Temperature Comparison", fontsize=13, fontweight="bold")
ax4.set_xticks(x_pos)
ax4.set_xticklabels(season_order)
ax4.grid(True, alpha=0.3, axis="y")
ax4.legend()

# Plot 5: Seasonal revenue
ax5 = axes[1, 1]
bars = ax5.bar(
    seasonal_metrics["season"],
    seasonal_metrics["total_revenue"],
    color=colors_season,
    edgecolor="black",
    linewidth=2,
)
ax5.set_ylabel("Total Revenue ($)", fontsize=12, fontweight="bold")
ax5.set_title("Seasonal Revenue", fontsize=13, fontweight="bold")
ax5.grid(True, alpha=0.3, axis="y")
for _i, bar in enumerate(bars):
    height = bar.get_height()
    ax5.text(
        bar.get_x() + bar.get_width() / 2.0,
        height,
        f"${height:.0f}",
        ha="center",
        va="bottom",
        fontsize=10,
        fontweight="bold",
    )

# Plot 6: Daily generation patterns by season
ax6 = axes[1, 2]
for season, color in zip(season_order, colors_season):
    season_hourly = df[df["season"] == season].groupby("hour")["power_output_kw"].mean()
    ax6.plot(
        season_hourly.index,
        season_hourly.values,
        linewidth=2.5,
        marker="o",
        markersize=4,
        label=season,
        color=color,
    )
ax6.set_xlabel("Hour of Day", fontsize=12, fontweight="bold")
ax6.set_ylabel("Avg Power (kW)", fontsize=12, fontweight="bold")
ax6.set_title("Daily Generation Patterns by Season", fontsize=13, fontweight="bold")
ax6.grid(True, alpha=0.3)
ax6.legend()
ax6.set_xlim(0, 23)

plt.tight_layout()
plt.show()

# Print seasonal comparison table
print("=" * 80)
print("SEASONAL PERFORMANCE COMPARISON")
print("=" * 80)
print(
    "\n{:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(
        "Season", "Energy (kWh)", "Capacity %", "Avg Irrad", "Panel Temp", "Revenue ($)"
    )
)
print("-" * 80)
for _, row in seasonal_metrics.iterrows():
    print(
        "{:<12} {:<12.0f} {:<12.1f} {:<12.0f} {:<12.1f} {:<12.2f}".format(
            row["season"],
            row["total_energy"],
            row["capacity_factor"],
            row["avg_irradiance"],
            row["avg_panel_temp"],
            row["total_revenue"],
        )
    )

print("\n" + "=" * 80)
print("KEY SEASONAL INSIGHTS")
print("=" * 80)
best_season = seasonal_metrics.loc[seasonal_metrics["total_energy"].idxmax(), "season"]
worst_season = seasonal_metrics.loc[seasonal_metrics["total_energy"].idxmin(), "season"]
seasonal_variation = seasonal_metrics["total_energy"].max() / seasonal_metrics["total_energy"].min()

print(f"\nBest Performing Season: {best_season}")
print(
    f"  • Energy: {seasonal_metrics.loc[seasonal_metrics['season'] == best_season, 'total_energy'].values[0]:.0f} kWh"
)
print(
    f"  • Capacity Factor: {seasonal_metrics.loc[seasonal_metrics['season'] == best_season, 'capacity_factor'].values[0]:.1f}%"
)
print(f"\nWorst Performing Season: {worst_season}")
print(
    f"  • Energy: {seasonal_metrics.loc[seasonal_metrics['season'] == worst_season, 'total_energy'].values[0]:.0f} kWh"
)
print(
    f"  • Capacity Factor: {seasonal_metrics.loc[seasonal_metrics['season'] == worst_season, 'capacity_factor'].values[0]:.1f}%"
)
print(f"\nSeasonal Variation: {seasonal_variation:.2f}x (best vs worst)")
print("\nTemperature Impact:")
print(
    f"  • Coolest season (Winter) avg panel temp: {seasonal_metrics.loc[seasonal_metrics['season'] == 'Winter', 'avg_panel_temp'].values[0]:.1f}°C"
)
print(
    f"  • Hottest season (Summer) avg panel temp: {seasonal_metrics.loc[seasonal_metrics['season'] == 'Summer', 'avg_panel_temp'].values[0]:.1f}°C"
)
print(
    f"  • Temperature differential impact on efficiency: ~{(seasonal_metrics.loc[seasonal_metrics['season'] == 'Summer', 'avg_panel_temp'].values[0] - seasonal_metrics.loc[seasonal_metrics['season'] == 'Winter', 'avg_panel_temp'].values[0]) * abs(TEMP_COEFFICIENT) * 100:.1f}% loss in summer"
)

## 10. Comprehensive Summary Dashboard

Final summary with all key performance indicators, economic metrics, and operational insights.

In [ ]:
# Create comprehensive summary dashboard

fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Large plot: Annual energy production timeline
ax_main = fig.add_subplot(gs[0, :])
daily_production = df.groupby("date")["power_output_kw"].sum()
daily_production.index = pd.to_datetime(daily_production.index)
ax_main.plot(
    daily_production.index, daily_production.values, linewidth=1, color="#3498DB", alpha=0.7
)
# Add 7-day rolling average
rolling_avg = daily_production.rolling(window=7, center=True).mean()
ax_main.plot(rolling_avg.index, rolling_avg.values, linewidth=3, color="#E74C3C", label="7-Day Avg")
ax_main.fill_between(daily_production.index, 0, daily_production.values, alpha=0.3, color="#3498DB")
ax_main.set_xlabel("Date", fontsize=12, fontweight="bold")
ax_main.set_ylabel("Daily Energy Production (kWh)", fontsize=12, fontweight="bold")
ax_main.set_title("Annual Solar Energy Production Timeline", fontsize=15, fontweight="bold")
ax_main.grid(True, alpha=0.3)
ax_main.legend(fontsize=11)

# Small plot 1: Energy metrics
ax1 = fig.add_subplot(gs[1, 0])
ax1.axis("off")
energy_summary = f"""
ENERGY METRICS
{"─" * 30}
Total Production:
  {total_energy_yield_kwh:,.0f} kWh/year

Specific Yield:
  {specific_yield:.0f} kWh/kWp/year

Capacity Factor:
  {capacity_factor:.2f}%

Performance Ratio:
  {performance_ratio:.2f}%

Daily Average:
  {total_energy_yield_kwh / 365:.1f} kWh/day

Peak Output:
  {df["power_output_kw"].max():.2f} kW
"""
ax1.text(
    0.1,
    0.9,
    energy_summary,
    transform=ax1.transAxes,
    fontsize=11,
    verticalalignment="top",
    fontfamily="monospace",
    bbox={"boxstyle": "round", "facecolor": "#E8F8F5", "alpha": 0.8},
)

# Small plot 2: Economic metrics
ax2 = fig.add_subplot(gs[1, 1])
ax2.axis("off")
economic_summary = f"""
ECONOMIC METRICS
{"─" * 30}
Annual Revenue:
  ${total_annual_revenue:,.2f}

LCOE:
  ${lcoe:.4f}/kWh

Payback Period:
  {simple_payback_years:.1f} years

25-Year ROI:
  {roi:.1f}%

Lifetime Profit:
  ${lifetime_profit:,.0f}

Monthly Avg:
  ${total_annual_revenue / 12:,.2f}
"""
ax2.text(
    0.1,
    0.9,
    economic_summary,
    transform=ax2.transAxes,
    fontsize=11,
    verticalalignment="top",
    fontfamily="monospace",
    bbox={"boxstyle": "round", "facecolor": "#E8F6F3", "alpha": 0.8},
)

# Small plot 3: Environmental metrics
ax3 = fig.add_subplot(gs[1, 2])
ax3.axis("off")
# Calculate CO2 avoided (typical grid: 0.92 lbs CO2/kWh = 0.417 kg/kWh)
co2_avoided_kg = total_energy_yield_kwh * 0.417
co2_avoided_tons = co2_avoided_kg / 1000
trees_equivalent = co2_avoided_tons * 16.5  # 1 tree absorbs ~60 kg CO2/year
cars_equivalent = co2_avoided_tons / 4.6  # Average car emits 4.6 tons/year

environmental_summary = f"""
ENVIRONMENTAL IMPACT
{"─" * 30}
CO₂ Avoided:
  {co2_avoided_tons:.1f} metric tons

Equivalent To:
  {trees_equivalent:.0f} trees planted

Cars Off Road:
  {cars_equivalent:.1f} cars/year

Coal Avoided:
  {co2_avoided_kg / 2.2:.0f} lbs

Homes Powered:
  {total_energy_yield_kwh / 10649:.1f} avg homes
  (annual basis)
"""
ax3.text(
    0.1,
    0.9,
    environmental_summary,
    transform=ax3.transAxes,
    fontsize=11,
    verticalalignment="top",
    fontfamily="monospace",
    bbox={"boxstyle": "round", "facecolor": "#E8F8E8", "alpha": 0.8},
)

# Small plot 4: Monthly production heatmap
ax4 = fig.add_subplot(gs[2, 0])
monthly_daily_avg = df.groupby(["month", df.index.day])["power_output_kw"].sum().reset_index()
monthly_daily_pivot = monthly_daily_avg.pivot_table(
    values="power_output_kw", index="datetime", columns="month"
)
sns.heatmap(
    monthly_daily_pivot.T,
    cmap="YlOrRd",
    cbar_kws={"label": "Daily Energy (kWh)"},
    ax=ax4,
    xticklabels=False,
)
ax4.set_xlabel("Day of Month", fontsize=11, fontweight="bold")
ax4.set_ylabel("Month", fontsize=11, fontweight="bold")
ax4.set_title("Daily Production Heatmap", fontsize=12, fontweight="bold")

# Small plot 5: Performance distribution
ax5 = fig.add_subplot(gs[2, 1])
metrics_names = ["Capacity\nFactor", "Performance\nRatio", "TOU\nAlignment", "System\nEfficiency"]
metrics_values = [
    capacity_factor,
    performance_ratio,
    peak_alignment,
    efficiency_data["power_efficiency"].mean(),
]
colors_metrics = [
    "#27AE60" if v >= 75 else "#F39C12" if v >= 60 else "#E74C3C" for v in metrics_values
]
bars = ax5.barh(metrics_names, metrics_values, color=colors_metrics, edgecolor="black")
ax5.set_xlabel("Percentage (%)", fontsize=11, fontweight="bold")
ax5.set_title("Key Performance Indicators", fontsize=12, fontweight="bold")
ax5.set_xlim(0, 100)
ax5.grid(True, alpha=0.3, axis="x")
for _i, bar in enumerate(bars):
    width = bar.get_width()
    ax5.text(
        width,
        bar.get_y() + bar.get_height() / 2.0,
        f"{width:.1f}%",
        ha="left",
        va="center",
        fontsize=10,
        fontweight="bold",
    )

# Small plot 6: System status
ax6 = fig.add_subplot(gs[2, 2])
ax6.axis("off")
status_summary = f"""
SYSTEM STATUS
{"─" * 30}
Overall Grade:
  {"A (Excellent)" if performance_ratio >= 80 else "B (Good)" if performance_ratio >= 75 else "C (Fair)"}

Health Indicators:
  ✓ Performance Ratio: {performance_ratio:.1f}%
  {"✓" if capacity_factor >= 18 else "⚠"} Capacity Factor: {capacity_factor:.1f}%
  {"✓" if peak_alignment >= 65 else "⚠"} TOU Alignment: {peak_alignment:.1f}%
  {"✓" if simple_payback_years <= 10 else "⚠"} Payback: {simple_payback_years:.1f} yrs

Optimization Score:
  {(performance_ratio + capacity_factor * 4 + peak_alignment) / 6:.0f}/100

Recommendations:
  • {"Panel cleaning" if performance_ratio < 80 else "Maintain schedule"}
  • {"Check inverters" if capacity_factor < 18 else "Operating optimally"}
  • Monitor degradation
"""
ax6.text(
    0.1,
    0.9,
    status_summary,
    transform=ax6.transAxes,
    fontsize=11,
    verticalalignment="top",
    fontfamily="monospace",
    bbox={"boxstyle": "round", "facecolor": "#FEF9E7", "alpha": 0.8},
)

# Add main title
fig.suptitle(
    "Solar PV System Performance Dashboard - Annual Summary",
    fontsize=18,
    fontweight="bold",
    y=0.995,
)

plt.show()

# Print final comprehensive summary
print("\n" + "=" * 80)
print("FINAL COMPREHENSIVE SUMMARY")
print("=" * 80)
print("\nSYSTEM OVERVIEW:")
print("  • 50 kW solar PV system operating for one full year")
print(f"  • Total data points: {len(df):,} hourly measurements")
print(f"  • System availability: {(df['power_output_kw'] > 0).sum() / len(df) * 100:.1f}%")

print("\nPERFORMANCE HIGHLIGHTS:")
print(f"  • Annual energy production: {total_energy_yield_kwh:,.0f} kWh")
print(f"  • Capacity factor: {capacity_factor:.2f}% (industry avg: 15-25%)")
print(f"  • Performance ratio: {performance_ratio:.2f}% (target: >75%)")
print(f"  • System efficiency: {efficiency_data['power_efficiency'].mean():.1f}%")

print("\nECONOMIC PERFORMANCE:")
print(f"  • Annual revenue: ${total_annual_revenue:,.2f}")
print(f"  • LCOE: ${lcoe:.4f}/kWh")
print(f"  • Simple payback: {simple_payback_years:.1f} years")
print(f"  • 25-year ROI: {roi:.1f}%")
print(f"  • Lifetime profit: ${lifetime_profit:,.0f}")

print("\nENVIRONMENTAL IMPACT:")
print(f"  • CO₂ emissions avoided: {co2_avoided_tons:.1f} metric tons/year")
print(f"  • Equivalent to planting {trees_equivalent:.0f} trees")
print(f"  • Equivalent to removing {cars_equivalent:.1f} cars from the road")

print("\nOPTIMIZATION INSIGHTS:")
print(f"  • {peak_alignment:.1f}% of generation occurs during peak price hours")
print(f"  • Time-of-use pricing adds ${tou_advantage:.2f} annual value")
print(f"  • Temperature losses: {df['temp_loss_kw'].sum():.0f} kWh/year")
print(f"  • Seasonal variation: {seasonal_variation:.2f}x (best vs worst)")

print("\nRECOMMENDATIONS:")
if performance_ratio >= 80:
    print("  ✓ System performing excellently - maintain current operations")
elif performance_ratio >= 75:
    print("  ✓ System performing well - consider minor optimization")
else:
    print("  ⚠ System performance below target - investigate losses")

if capacity_factor >= 20:
    print("  ✓ Capacity factor above industry average")
elif capacity_factor >= 15:
    print("  ✓ Capacity factor within industry range")
else:
    print("  ⚠ Capacity factor below industry average - check for issues")

print("\n  • Schedule regular panel cleaning to maintain efficiency")
print("  • Monitor inverter performance monthly")
print("  • Track degradation rate annually")
print("  • Consider battery storage to capture off-peak generation")

print("\n" + "=" * 80)
print("Analysis complete! This solar PV system demonstrates the technical and")
print("economic viability of renewable energy, with strong performance metrics,")
print("positive ROI, and significant environmental benefits.")
print("=" * 80)